In [44]:
import numpy as np
from deepsy.core.model import Model
from deepsy.core.nn import NeuralNetwork
from deepsy.core.layer import Layer
from deepsy.core.functions.activation_functions import *
from deepsy.core.functions.loss_functions import *
from deepsy.core.functions.optimizers import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [45]:
dataset = pd.read_csv('data/binary/titanic.csv')
Y = dataset['Survived']
X = dataset.drop(columns=['Survived'])
X = X.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
X = pd.get_dummies(X, columns=['Pclass', 'Sex', 'Embarked'])

X['Age'].fillna(X['Age'].mean(), inplace=True)
X['Fare'].fillna(X['Fare'].mean(), inplace=True)

X = (X - X.mean()) / X.std()

X, Y = X.values, Y.values

print('Dataset shape: {}'.format(dataset.shape))
print(dataset.head())
print()
print(X[:5])
print()
print(Y[:5])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

nn = NeuralNetwork(layers=[
    Layer(nr_neurons=12, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=16, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=8, nr_in_features=16, activation_func=ReLU()),
    Layer(nr_neurons=1, nr_in_features=8, activation_func=Sigmoid())
])

model = Model(nn, loss_func=BinaryCrossEntropy(), optimizer=GradientDescent(0.4))
model.train(X_train.T, Y_train.T, 400)

Dataset shape: (891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8

In [46]:
Y_predicted = model.predict(X_test.T)
acc = sum((Y_predicted.reshape(Y_test.shape) > 0.5) == Y_test) / len(Y_test)
print('Accuracy: {}'.format(acc))
for i in range(len(Y_predicted[0])):
    print('predicted: {}   actual: {}'.format(Y_predicted[0][i], Y_test[i]))

Accuracy: 0.8044692737430168
predicted: 0.2793650780226103   actual: 0
predicted: 0.1305207497649125   actual: 0
predicted: 0.2585571428932372   actual: 1
predicted: 0.12863074108312159   actual: 0
predicted: 0.8832056635101727   actual: 1
predicted: 0.6040236126102786   actual: 0
predicted: 0.09313259195748788   actual: 0
predicted: 0.5409171593996275   actual: 1
predicted: 0.5288002548287529   actual: 0
predicted: 0.12344192879905193   actual: 0
predicted: 0.1161479826907718   actual: 0
predicted: 0.16792272616445036   actual: 0
predicted: 0.5555859279828113   actual: 1
predicted: 0.4519803907988181   actual: 0
predicted: 0.14220485356396836   actual: 1
predicted: 0.15822943122676994   actual: 0
predicted: 0.1199939480753272   actual: 0
predicted: 0.15302676183832029   actual: 0
predicted: 0.11552708976080764   actual: 0
predicted: 0.16927602464377928   actual: 0
predicted: 0.2778060598432093   actual: 0
predicted: 0.5222396858099418   actual: 0
predicted: 0.08990406516167539   actua

In [47]:
dataset = pd.read_csv('data/binary/Employee.csv')
Y = dataset['LeaveOrNot']
X = dataset.drop(columns=['LeaveOrNot'])
X = pd.get_dummies(X, columns=['Education', 'City', 'Gender', 'EverBenched'])

X = (X - X.mean()) / X.std()

X, Y = X.values, Y.values

print('Dataset shape: {}'.format(dataset.shape))
print(dataset.head())
print()
print(X[:5])
print()
print(Y[:5])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

nn = NeuralNetwork(layers=[
    Layer(nr_neurons=12, nr_in_features=14, activation_func=ReLU()),
    Layer(nr_neurons=16, nr_in_features=12, activation_func=ReLU()),
    Layer(nr_neurons=8, nr_in_features=16, activation_func=ReLU()),
    Layer(nr_neurons=1, nr_in_features=8, activation_func=Sigmoid())
])

model = Model(nn, loss_func=BinaryCrossEntropy(), optimizer=GradientDescent(1))
model.train(X_train.T, Y_train.T, 500)

Dataset shape: (4653, 9)
   Education  JoiningYear       City  PaymentTier  Age  Gender EverBenched  \
0  Bachelors         2017  Bangalore            3   34    Male          No   
1  Bachelors         2013       Pune            1   28  Female          No   
2  Bachelors         2014  New Delhi            3   38  Female          No   
3    Masters         2016  Bangalore            3   27    Male          No   
4    Masters         2017       Pune            3   24    Male         Yes   

   ExperienceInCurrentDomain  LeaveOrNot  
0                          0           0  
1                          3           1  
2                          2           0  
3                          5           1  
4                          2           1  

[[ 1.03952665  0.53744523  0.95454254 -1.86470098  0.54044268 -0.48052341
  -0.20000086  1.04316163 -0.57522042 -0.61197491 -0.82146269  0.82146269
   0.33832885 -0.33832885]
 [-1.10711376 -3.024852   -0.28870069  0.06054761  0.54044268 -0.4805234

In [48]:
Y_predicted = model.predict(X_test.T)
acc = sum((Y_predicted.reshape(Y_test.shape) > 0.5) == Y_test) / len(Y_test)
print('Accuracy: {}'.format(acc))
for i in range(len(Y_predicted[0])):
    print('predicted: {}   actual: {}'.format(Y_predicted[0][i], Y_test[i]))

Accuracy: 0.832438238453276
predicted: 0.06873078637975763   actual: 0
predicted: 0.018537184034213297   actual: 0
predicted: 0.2160093516170745   actual: 0
predicted: 0.004297116207374617   actual: 0
predicted: 0.09686889796493738   actual: 0
predicted: 0.05263825302998656   actual: 0
predicted: 0.13419071271686495   actual: 0
predicted: 0.0341731398823476   actual: 1
predicted: 0.47746107549180417   actual: 0
predicted: 0.1303386688744353   actual: 0
predicted: 0.01577589442732827   actual: 0
predicted: 0.9912292321297935   actual: 1
predicted: 0.06815241060269889   actual: 0
predicted: 0.34383458166896635   actual: 1
predicted: 0.20518234195183388   actual: 0
predicted: 0.9997194267437562   actual: 1
predicted: 0.08161905918466349   actual: 0
predicted: 0.02068695838193308   actual: 0
predicted: 0.014947731771930154   actual: 0
predicted: 0.9998795357510994   actual: 1
predicted: 0.39565275018430573   actual: 1
predicted: 0.02543754385145364   actual: 0
predicted: 0.99780354054501  

In [49]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, (Y_predicted.reshape(Y_test.shape) > 0.5)))

              precision    recall  f1-score   support

           0       0.81      0.98      0.88       610
           1       0.92      0.56      0.70       321

    accuracy                           0.83       931
   macro avg       0.87      0.77      0.79       931
weighted avg       0.85      0.83      0.82       931

